# 3D toy model

- generate a segmented 3d image of cells
- modify image to create explicit membrane compartments
- combine cells, combine membranes
- create a simple sme model using this geometry
- do an example simulation

### Navigation

- Press `Space` to show the next page
- Press `Shift+Space` to show the previous page
- Press `Escape` to zoom out

### Utility functions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from mpl_toolkits.mplot3d import Axes3D
import cv2
import imageio.v3 as iio
import tifffile
import skimage
import sme
from IPython.display import HTML
from scipy import ndimage as ndi

plt.rcParams["figure.figsize"] = (8, 8)

In [ ]:
def show_image(image, title=""):
    if image.ndim == 3:
        img = skimage.color.label2rgb(image)
    else:
        img = image / np.max(image)
    depth, rows, cols, nc = img.shape
    x = np.arange(rows)
    y = np.arange(cols)
    x, y = np.meshgrid(x, y)

    fig = plt.figure()
    ax = fig.add_subplot(111, projection="3d")
    for i in range(depth):
        ax.plot_surface(
            x,
            y,
            i * np.ones((cols, rows)),
            facecolors=img[i],
            rstride=1,
            cstride=1,
            alpha=0.5,
            shade=False,
            linewidth=0,
        )
    plt.title(title)
    plt.show()

In [ ]:
def sphere_mask(grid_shape, center, radius, deformation):
    X, Y, Z = grid_shape
    x0, y0, z0 = center
    dx, dy, dz = deformation
    x, y, z = np.ogrid[:X, :Y, :Z]
    return dx * (x - x0) ** 2 + dy * (y - y0) ** 2 + dz * (z - z0) ** 2 <= radius**2

In [ ]:
def spheres(n_pixels, n_spheres, max_radius, max_deform):
    voxels = np.zeros((n_pixels, n_pixels, n_pixels), dtype=np.uint8)
    for n_sphere in range(1, n_spheres + 1):
        center = np.random.randint(2, n_pixels - 2, 3)
        nuclear_radius = np.random.randint(1, max_radius / 2)
        cell_radius = np.random.randint(1.5 * nuclear_radius, max_radius)
        deformation = np.random.uniform(1 / max_deform, max_deform, 3)
        voxels[
            sphere_mask(voxels.shape, center, nuclear_radius, deformation)
        ] = n_sphere
    return voxels

### Generate segmented input data

- construct a 40x40x40 3d image with 50 randomly distributed, sized and deformed spheres
- each voxel has an index which identifies which sphere (if any) it belongs to

In [ ]:
img_indexed = spheres(n_pixels=40, n_spheres=50, max_radius=15, max_deform=1.5)

In [ ]:
show_image((img_indexed), "Segmented cells input image")

### Generate explicit membranes by dilating each cell

- We want to add explicit membrane compartments around each cell.
- To do this we take a mask of each cell individually, dilate it, and select the pixels that differ from the original mask
- Repeating this over all cells and combining the results gives us a mask of membrane compartment pixels

In [ ]:
img_membrane_mask = np.zeros(img_indexed.shape).astype(bool)
kernel = ndi.generate_binary_structure(rank=3, connectivity=1)
kernel_size = (3, 3, 3)
kernel = np.ones(kernel_size, dtype=np.uint8)
for index in range(img_indexed.max()):
    img = (img_indexed == index).astype(np.uint8)
    img_membrane_mask |= ndi.binary_dilation(img) != img

In [ ]:
show_image(img_membrane_mask, "Membrane pixels mask")

### Define cells as any segmented pixel excluding membrane pixels

- Now we select all pixels that were identified as cells
- Then we exclude pixels that are part of the membrane mask to leave a cell mask

In [ ]:
img_cell_mask = img_indexed != 0
img_cell_mask = img_cell_mask & (img_cell_mask != img_membrane_mask)

In [ ]:
show_image(img_cell_mask, "Cell pixels mask")

### Construct segmented geometry image for sme

- From these masks we can construct a segmented geometry image for sme
- Each colour in this image can then be assigned to a compartment in the model

In [ ]:
img = np.zeros(img_cell_mask.shape, dtype=np.uint8)
img[img_cell_mask] = 1
img[img_membrane_mask] = 2
tifffile.imwrite("geom3d.tiff", img)

In [ ]:
show_image(img, "Segmented geometry image for sme")

### Create sme model

- This was done using the GUI, starting from the 2d toy model & importing the 3d geomtery image
- As in the 2d case: one species in each compartment, intially only non-zero in outside
- Reactions: `outside <-> membrane` and `membrane <-> cell`

In [ ]:
model = sme.Model("3d-toy-model.xml")

In [ ]:
model.compartment_image.shape

In [ ]:
model.import_geometry_from_image("geom3d.tiff")

In [ ]:
model.compartment_image.shape

In [ ]:
show_image(model.compartment_image, "Model compartment geometry")

### Simulate model

- simulate for 600s, storing the results every 300s
- this might take a few minutes

In [ ]:
simulation_results = model.simulate(600, 300)

### Simulation results

In [ ]:
show_image(
    simulation_results[0].concentration_image / 255.0,
    f"Concentrations at t={simulation_results[0].time_point}",
)

In [ ]:
show_image(
    simulation_results[1].concentration_image / 255.0,
    f"Concentrations at t={simulation_results[1].time_point}",
)

In [ ]:
show_image(
    simulation_results[2].concentration_image / 255.0,
    f"Concentrations at t={simulation_results[2].time_point}",
)